In [6]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import numpy as np
import scipy.sparse as sps
from DataReader import DataReaderChallangePolimi2017
dr = DataReaderChallangePolimi2017(evaluation = False, rebuild = False)

urm = dr.getURM_csr()
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
test = dr.getTestURM_csr()

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderFullDump
Dataset full loaded
--- !!!Warning!!! There is not test set here! ---


In [7]:
urm_plus = sps.vstack([urm,icm.T]).tocsr()

In [8]:
recommender = SLIM_BPR_Cython(urm_plus, recompile_cython=False, positive_threshold=1, sparse_weights=True )
logFile = open("SLIM_BPR_Cython2.txt", "a")
recommender.fit(epochs=10, validate_every_N_epochs= 1000, URM_test=test,topK=100,
                logFile=logFile, batch_size=1, sgd_mode='adagrad', learning_rate=1e-4,
                 start_validation_after_N_epochs = 1000,
                 lambda_i = 0.0025, lambda_j = 0.0025,  # lambda_i = 0.0025, lambda_j = 0.00025,
                )

sim = recommender.getSimilarity()
#dr.evaluateMAP(urm * sim)

Epoch 0 of 10 complete in 0.01 minutes
Processed 500000 ( 41.19% ) in 12.92 seconds. Sample per second: 38706
Processed 1000000 ( 82.39% ) in 10.12 seconds. Sample per second: 45200
Processed 1213766 ( 100.00% ) in 3.68 seconds. Sample per second: 47267
Return S matrix to python caller
Epoch 1 of 10 complete in 0.52 minutes
Processed 500000 ( 41.19% ) in 6.57 seconds. Sample per second: 76057
Processed 1000000 ( 82.39% ) in 7.92 seconds. Sample per second: 71819
Processed 1213766 ( 100.00% ) in 4.24 seconds. Sample per second: 70392
Return S matrix to python caller
Epoch 2 of 10 complete in 0.41 minutes
Processed 500000 ( 41.19% ) in 7.75 seconds. Sample per second: 64541
Processed 1000000 ( 82.39% ) in 8.75 seconds. Sample per second: 63488
Processed 1213766 ( 100.00% ) in 4.19 seconds. Sample per second: 63233
Return S matrix to python caller
Epoch 3 of 10 complete in 0.44 minutes
Processed 500000 ( 41.19% ) in 6.46 seconds. Sample per second: 77388
Processed 1000000 ( 82.39% ) in 8.

In [9]:
sim = recommender.getSimilarity()

In [10]:
dr.save_sparse_csr(array=sim,filename="./DumpData/sim_bpr_10e_100k_fulltrain")